## Inception 
![](./imgs/inception.svg)

In [1]:
from torch import nn
from torch.nn import functional as F
import torch
class Inception(nn.Module):
    def __init__(self,in_channels,out_channels):
        super().__init__()
        self.net_0=nn.Sequential(
            nn.Conv2d(in_channels,out_channels[0],kernel_size=1),
            nn.ReLU()
        )
        self.net_1=nn.Sequential(
            nn.Conv2d(in_channels,out_channels[1][0],kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(out_channels[1][0],out_channels[1][1],kernel_size=3,padding=1),
            nn.ReLU(),
        )
        self.net_2=nn.Sequential(
            nn.Conv2d(in_channels,out_channels[2][0],kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(out_channels[2][0],out_channels[2][1],kernel_size=5,padding=2),
            nn.ReLU(),
        )
        self.net_3=nn.Sequential(
            nn.MaxPool2d(kernel_size=3,padding=1,stride=1),
            nn.Conv2d(in_channels,out_channels[3],kernel_size=1),
            nn.ReLU(),
        )        
        
        
    
    
    def forward(self,X):
        X0=self.net_0(X)
        X1=self.net_1(X)
        X2=self.net_2(X)
        X3=self.net_3(X)
        return torch.cat((X0,X1,X2,X3),dim=1)
        


![](./imgs/inception-full.svg)

In [2]:
class GoogleNet(nn.Module):
    def __init__(self):
        super().__init__()
        net_0=nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        net_1=nn.Sequential(
            
            nn.Conv2d(64, 64, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        net_2=nn.Sequential(
            Inception(192,[64, (96, 128), (16, 32), 32]),
            Inception(256,[128, (128, 192), (32, 96), 64]),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        net_3=nn.Sequential(
            Inception(480, [192, (96, 208), (16, 48), 64]),
            Inception(512, [160, (112, 224), (24, 64), 64]),
            Inception(512, [128, (128, 256), (24, 64), 64]),
            Inception(512, [112, (144, 288), (32, 64), 64]),
            Inception(528, [256, (160, 320), (32, 128), 128]),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        net_4=nn.Sequential(
            Inception(832,[ 256, (160, 320), (32, 128), 128]),
            Inception(832,[ 384, (192, 384), (48, 128), 128]),
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten()
        )
        self.net=nn.Sequential(
            net_0,net_1,net_2,net_3,net_4,nn.Linear(1024, 10)
        )
        
    def forward(self,X):
        return self.net(X)

    





In [3]:
X = torch.rand(size=(1, 1, 96, 96))

In [4]:
import torch
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms
aug=transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([96,96]),
    transforms.ToTensor()]
)

class FashionMnist(Dataset):

    def __init__(self, img_path, label_path):
        super().__init__()
        self.load_data_from_path(img_path, label_path)

    def __getitem__(self, index):
        return self.imgs[index], self.labels[index]

    def __len__(self):
        return self.img_num

    def load_data_from_path(self, img_path, label_path):
        with open(img_path, 'rb') as f:
            s = f.read()

        self.img_num = int(s[4:8].hex(), 16)
        self.imgs = torch.FloatTensor(list(iter(s[16:])))
        # print(self.img_num,self.imgs)
        self.imgs = torch.reshape(self.imgs, (-1, 1, 28, 28))
        # print(self.imgs.shape)
        with open(label_path, 'rb') as f:
            s = f.read()
        self.labels = torch.tensor(list(iter(s[8:])))
        # print(self.labels.shape)



def convert_to_227(img,aug):
    return aug(img).numpy().tolist()


def collate_fn(batch):
    inputs, labels = [], []
    for X, y in batch:
        inputs.append(convert_to_227(X, aug))
        labels.append(y)
        
    
    return torch.Tensor(inputs), torch.tensor(labels)

train_img_path='D:/datasets/FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte'
train_label_path='D:/datasets/FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte'
test_img_path='D:/datasets/FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte'
test_label_path='D:/datasets/FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte'

train_data=FashionMnist(train_img_path,train_label_path)
test_data=FashionMnist(test_img_path,test_label_path)
train_iter=DataLoader(train_data,batch_size=16,shuffle=True,collate_fn=collate_fn)
test_iter=DataLoader(test_data,batch_size=16,shuffle=True,collate_fn=collate_fn)

In [5]:
def precise(test_iter,net,device):
    total=0
    correct=0
    with torch.no_grad():
        for X,y in test_iter:
            total+=len(X)
            X,y=X.to(device),y.to(device)
            y_hat=net(X)
            out_put=torch.argmax(y_hat,dim=-1)
            ans=(out_put==y)
            correct+=ans.sum().item()
        
    print(correct)
    print(f"accuracy :{correct/total*100:>3f}% ")

    
def train(data_iter,entroy_iter,net,optimizer,lr_scheduler,loss_fn,epochs,device,epoch_data_num):
    matrix_x,matrix_loss,entroy_loss,entroy_x=[0],[0],[],[]
    total_loss=0
    batchs=len(data_iter)
    for epoch in range(epochs):
        now_num=0
        for X,y in data_iter:
 
            now_num+=len(X)
    
            net.train()
            
            X,y=X.to(device),y.to(device)
            optimizer.zero_grad()
            y_hat=net(X)
            loss=loss_fn(y_hat,y)
            loss.sum().backward()

            optimizer.step()

            total_loss+=loss.item()

            matrix_x.append(matrix_x[-1]+1)
            matrix_loss.append(total_loss/(epoch*epoch_data_num+now_num))
            
            print(f"loss: {matrix_loss[-1]:>7f} now {matrix_x[-1]}/{batchs*epochs}",end='\r')
            
        
        lr_scheduler.step()
        with torch.no_grad():
            c_total_loss=0
            test_data_num=0
            for X,y in entroy_iter: 
                net.eval()
                test_data_num+=len(X)
                X,y=X.to(device),y.to(device)
                y_hat=net(X)
                loss=loss_fn(y_hat,y)
                
                c_total_loss+=loss.item()
            
            print(test_data_num)
            entroy_loss.append(c_total_loss/test_data_num)
            entroy_x.append((epoch+1)*batchs)
        print(f"cross entroy loss:{entroy_loss[-1]} now {epoch+1}/{epochs}")
        torch.save(net.state_dict(), f"Google_epoch{epoch+6}.bin")
        precise(test_iter,net,device)

  
    
    return net,matrix_x,matrix_loss,entroy_x,entroy_loss



In [ ]:
from torch.optim import Adam
from torch.optim.lr_scheduler import LambdaLR
device='cuda' if torch.cuda.is_available() else 'cpu'
net=GoogleNet().to(device)
net.load_state_dict(torch.load('./Google_epoch5.bin'))
optimizer=Adam(net.parameters(),lr=0.000016)
lr_scheduler=LambdaLR(optimizer, lr_lambda=lambda epoch: 1/(2**epoch))
loss_fn=nn.CrossEntropyLoss()
'''for m in net.modules():
    if isinstance(m, (nn.Conv2d, nn.Linear)):
        nn.init.xavier_uniform_(m.weight)
'''
_,matrix_x,matrix_loss,entroy_x,entroy_loss=train(train_iter,test_iter,net,optimizer,lr_scheduler,loss_fn,4,device,len(train_data))

10000 0.015182 now 3750/15000
cross entroy loss:0.019338746465323492 now 1/4
8883
accuracy :88.830000% 
10000 0.014961 now 7500/15000
cross entroy loss:0.019210604816582055 now 2/4
8889
accuracy :88.890000% 
10000 0.014814 now 11250/15000
cross entroy loss:0.019152787777595222 now 3/4
8888
accuracy :88.880000% 
